In [1]:
import pandas as pd
import numpy as np
import time
import psycopg2
import pickle

In [2]:
ubuntu=True

con = None

if ubuntu:
    con = psycopg2.connect(database='mountainproject', password='kepler31', user='devin', port=5432, host='/var/run/postgresql/')
else:
    con = psycopg2.connect(database='mountainproject', user='User')
cur = con.cursor()
con.autocommit = True

In [3]:

def get_user_id_gte(table='stars', n=5):
    #user_ids from users greater than or equal to n reviews
    user_id_gte= '''
        SELECT user_id
        FROM {}
        GROUP BY user_id
        HAVING COUNT(user_id) >= %s;
        '''.format(table)
    cur.execute(user_id_gte, (n,) )
    tup_list = cur.fetchall()
    #return tup_list
    return [tup[0] for tup in tup_list]
#user_id_list = get_user_id_gte('stars', 5), len(user_id_list)


In [4]:
def get_climb_id_from_user_id_gte(id_tuple, n, table='stars'):
    climb_ids = '''
        SELECT climb_id
        FROM {}
        WHERE user_id IN %s
        GROUP BY climb_id
        HAVING COUNT(climb_id) >= %s;
    '''.format(table)
    cur.execute(climb_ids, (id_tuple, n) )
    return [tup[0] for tup in cur.fetchall()]
#climb_id_list = get_climb_id_from_user_id_gte(tuple(user_id_list), 5), len(climb_id_list)


In [5]:
def get_user_id_from_climb_id_gte(id_tuple, n, table='stars'):
    #Get user ids with at least n reviews in the climb id tuple
    user_ids_sql = '''
        SELECT user_id
        FROM {}
        WHERE climb_id IN %s
        GROUP BY user_id
        HAVING COUNT(user_id) >= %s;
    '''.format(table)
    cur.execute(user_ids_sql, (id_tuple, n) )
    return [tup[0] for tup in cur.fetchall()]
#user_id_list2 = get_user_id_from_climb_id_gte(tuple(climb_id_list), 5), len(user_id_list2)


In [6]:
def find_climb_user_ids(table='stars', n=5):
    climb_id_old = [0]
    climb_id_new = [-2]
    user_id_old = [0]
    user_id_new = [-2] 
    user_id_new = get_user_id_gte(table=table, n=n)
    while (set(user_id_old) != set(user_id_new)) and \
            (set(climb_id_old) != set(climb_id_new)):
        print "climb list:", len(climb_id_new)
        print "user list:", len(user_id_new)
        climb_id_old = climb_id_new
        climb_id_new = get_climb_id_from_user_id_gte(tuple(user_id_new), 
                                                     n,
                                                     table=table)
        user_id_old = user_id_new
        user_id_new = get_user_id_from_climb_id_gte(tuple(climb_id_new),
                                                    n,
                                                    table=table)
        
    return user_id_new, climb_id_new

#(user_id_list, climb_id_list) = find_climb_user_ids(table='stars', n=5)

In [7]:
def pickle_id_list(id_list, name):
    # Save id list for future use
    with open('id_list_'+name+'.p','w') as f:
        pickle.dump(id_list, f)

In [18]:
(user_id_list, climb_id_list) = find_climb_user_ids(n=5)

climb list: 1
user list: 7592
climb list: 13694
user list: 7565
climb list: 13678
user list: 7562
climb list: 13676
user list: 7561


In [98]:
pickle_id_list(climb_id_list, 'climb_star5')
pickle_id_list(user_id_list, 'user_star5')

In [55]:
(user_id_list_sub, climb_id_list_sub) = find_climb_user_ids(n=30)

climb list: 1
user list: 2558
climb list: 2508
user list: 1555
climb list: 2042
user list: 1415
climb list: 1931
user list: 1367
climb list: 1897
user list: 1352
climb list: 1884
user list: 1350
climb list: 1879
user list: 1344
climb list: 1871
user list: 1337
climb list: 1864
user list: 1331
climb list: 1860
user list: 1329
climb list: 1857
user list: 1327
climb list: 1854
user list: 1326


In [56]:
pickle_id_list(climb_id_list_sub, 'climb_star30')
pickle_id_list(user_id_list_sub, 'user_star30')

In [96]:
(user_id_list_sub, climb_id_list_sub) = find_climb_user_ids(n=35)

climb list: 1
user list: 2255
climb list: 1940
user list: 1190
climb list: 1423
user list: 948
climb list: 1201
user list: 839
climb list: 1082
user list: 773
climb list: 1016
user list: 740
climb list: 983
user list: 724
climb list: 973
user list: 718
climb list: 964
user list: 716
climb list: 959
user list: 712
climb list: 951
user list: 706
climb list: 944
user list: 702
climb list: 942
user list: 701
climb list: 940
user list: 700


In [97]:
pickle_id_list(climb_id_list_sub, 'climb_star35')
pickle_id_list(user_id_list_sub, 'user_star35')

##Rebels: Select Data Where User is different from consensus

In [12]:
q = '''
SELECT user_id,
        stars.user_name,
       stars.climb_name,
       stars.climb_id,
       user_stars
FROM stars
JOIN tb_climb
ON tb_climb.climb_id=stars.climb_id
WHERE user_stars > (1.5+stars)
OR user_stars < (0.5 + stars)
LIMIT 5;
'''
cur.execute(q)
cur.fetchone()

(107942262, 'caesar.salad', 'Captain Fingers', 105908771, 5)

In [10]:
q = '''
select * from stars
limit 5;
'''
cur.execute(q)
cur.fetchone()

(111, 'some climb', 'some dude', 222, 4)

In [79]:
def reset_rebels(cur, con):
    #print "You don't want to do this."
    #return
    try:
        cur.execute("DROP TABLE rebels")
        print 'rebels table dropped'
    except:
        print 'didnt drop'
    try:
        cur.execute('CREATE TABLE rebels(user_id INTEGER, \
                                       climb_id INT,\
                                       user_stars INT)')
        

        print 'rebels table created'
        
    except:
        print 'didnt create'
        
    q = '''
    SELECT user_id,
           stars.climb_id,
           user_stars
    FROM stars
    JOIN tb_climb
    ON tb_climb.climb_id=stars.climb_id
    WHERE user_stars > (1.75+stars)
    OR user_stars < (0.25 + stars)
    ;
    '''
    cur.execute(q)
    
    for val in cur.fetchall():
        cur.execute("INSERT INTO rebels VALUES"+str(val)
                    )     

    con.commit()
reset_rebels(cur, con)

rebels table dropped
rebels table created


In [80]:
(user_id_list, climb_id_list) = find_climb_user_ids(n=5, table='rebels')

climb list: 1
user list: 3530
climb list: 3896
user list: 2957
climb list: 3686
user list: 2913


In [81]:
pickle_id_list(climb_id_list, 'climb_rebel5')
pickle_id_list(user_id_list, 'user_rebel5')

In [82]:
(user_id_list, climb_id_list) = find_climb_user_ids(n=10, table='rebels')

climb list: 1
user list: 1895
climb list: 1350
user list: 917
climb list: 926
user list: 722
climb list: 747
user list: 614
climb list: 666
user list: 551
climb list: 606
user list: 510
climb list: 573
user list: 484
climb list: 551
user list: 463
climb list: 529
user list: 452
climb list: 514
user list: 438
climb list: 493
user list: 418
climb list: 473
user list: 409
climb list: 465
user list: 403
climb list: 458
user list: 399
climb list: 455
user list: 397
climb list: 453
user list: 393
climb list: 445
user list: 386
climb list: 437
user list: 378
climb list: 425
user list: 371
climb list: 419
user list: 363
climb list: 409
user list: 357
climb list: 400
user list: 351
climb list: 391
user list: 340
climb list: 378
user list: 329
climb list: 362
user list: 314
climb list: 348
user list: 300
climb list: 336
user list: 289
climb list: 325
user list: 280
climb list: 312
user list: 272
climb list: 310
user list: 269
climb list: 309
user list: 267
climb list: 308
user list: 265
climb li

In [83]:
pickle_id_list(climb_id_list, 'climb_rebel10')
pickle_id_list(user_id_list, 'user_rebel10')

###Generate Sparse Matrix for Analysis

In [114]:
from scipy import sparse as sp

In [121]:
pickle_string = '' #'star35'

In [122]:
with open('id_list_climb_'+pickle_string+'.p','r') as f:
        climb_ids = pickle.load(f)
len(climb_ids)

299

In [123]:
with open('id_list_user_'+pickle_string+'.p','r') as f:
        user_ids = pickle.load(f)
len(user_ids)

260

In [124]:
user_set = set(user_ids)
climb_set = set(climb_ids)
climb_tuple = tuple(climb_ids)

#Remove Orphaned ID number.  Orphaned has many orphaned data points
if 105990845 in user_ids: user_ids.remove(105990845)

climb_dict = {}
for i, el in enumerate(climb_tuple):
    climb_dict[el]=i

data = []
row = []
col = []
data_dict = {}
user_ids_csv = []
climb_ids_csv = []
table = 'stars'
for i, user in enumerate(user_ids):
    #print user
    q = '''
    SELECT user_stars, climb_id
    FROM {}
    WHERE user_id = %s
    AND climb_id IN %s
    '''.format(table)
    cur.execute(q, (user, climb_tuple) )
    for el in cur.fetchall():
        data += [el[0]]
        
        user_ids_csv += [user]
        row += [i]
        
        climb_ids_csv += [el[1]]        
        col += [climb_dict[el[1]]]
        
        data_dict[(i,climb_dict[el[1]])] = el[0]
    
sp_mat_input = (data,(row,col))

In [125]:
#Write to csv
df = pd.DataFrame({'User':user_ids_csv,
                   'Climb':climb_ids_csv,
                   'rating':data})
df.to_csv(pickle_string+'.csv', index=False)
df

,Climb,User,rating
0,105989461,107860315,5
1,105857287,107860315,4
2,105983489,107860315,5
3,105762201,107860315,4
4,105753838,107860315,4
5,105753754,107860315,4
6,105753760,107860315,4
7,105753391,107860315,4
8,105753397,107860315,5
9,105753394,107860315,3


In [126]:
q = '''
    SELECT DISTINCT climb_id, yds, yds_consensus, stars, star_votes, location, type_text
    FROM tb_climb
    WHERE climb_id IN %s
'''
cur.execute(q, (climb_tuple,))

In [127]:
df = pd.DataFrame(cur.fetchall())
df.head()

,0,1,2,3,4,5,6
0,105741011,5.7,5.7,2.8,164,"Idaho,City of Rocks,Breadloaves,Bloody Fingers...","Trad, 1 pitch, 110'"
1,105822918,5.7,5.7,3.1,102,"Utah,Wasatch Range,Little Cottonwood Canyon,Li...","Trad, 1 pitch"
2,105721840,5.9,5.9,2.8,80,"California,Joshua Tree National Park,Wonderlan...","Trad, 1 pitch, 160'"
3,105858078,5.10c,5.10c,3.2,57,"International,North America,Canada,British Col...","Sport, 70'"
4,105732260,5.9,5.9,3.1,154,"Nevada,Red Rock,Juniper Canyon,Ginger Buttress","Trad, 7 pitches, 900', Grade III"


In [128]:
#Mapping YDS to South Africa Ewbank because it is purely numeric
rating_map={'4th': 2,
            '5th': 3.5,
            '5.0': 3.5,
            '5.1': 5.5,
            '5.2': 7.5,
            '5.3': 8.5,
            '5.4': 10.5,
            '5.5': 11.5,
            '5.6': 13,
            '5.7': 14.5,
            '5.7+': 15,
            '5.8-': 15.5,
            '5.8': 16,
            '5.8+': 16.5,
            '5.9-': 17,
            '5.9': 17.5,
            '5.9+': 18,
            '5.10': 20,
            '5.10+': 21,
            '5.10-': 19.5,
            '5.10a': 19,
            '5.10a/b': 19.5,
            '5.10b': 20,
            '5.10b/c': 20.5,
            '5.10c': 21,
            '5.10c/d': 21.5,
            '5.10d': 22,
            '5.11': 23.5,
            '5.11+': 24.5,
            '5.11-': 23,
            '5.11a': 22.5,
            '5.11a/b':22.75,
            '5.11b': 23,
            '5.11b/c': 23.5,
            '5.11c': 24,
            '5.11c/d': 24.5,
            '5.11d': 25,
            '5.12': 27.5,
            '5.12+': 28.5,
            '5.12-': 26.5,
            '5.12a': 26,
            '5.12a/b': 26.5,
            '5.12b': 27,
            '5.12b/c': 27.5,
            '5.12c': 28,
            '5.12c/d': 28.5,
            '5.12d': 29,
            '5.13': 31.5,
            '5.13+': 32.5,
            '5.13-': 30.5,
            '5.13a': 30,
            '5.13a/b': 30.5,
            '5.13b': 31,
            '5.13b/c': 31.5,
            '5.13c': 32,
            '5.13c/d': 32.5,
            '5.13d': 33,
            '5.14-': 34.5,
            '5.14a': 34,
            '5.14a/b': 34.5,
            '5.14b': 35,
            '5.14c': 36,
            '5.14d': 37,
            '5.15a': 38}

In [129]:
qr = '''
    select distinct yds_consensus
    from tb_climb
    '''
cur.execute(qr)
yds_unique = sorted([yds[0] for yds in cur.fetchall()])
set(yds_unique) - set(rating_map.keys())

set()

In [130]:
climb_observed_features = pd.DataFrame({'Climb': df.loc[:,0],
                                        'rating': [rating_map[yds] for yds in df.loc[:,1] ],
                                        'rating_dif': np.array([rating_map[yds] for yds in df.loc[:,1] ]) -\
                                       np.array([rating_map[yds] for yds in df.loc[:,2]]),
                                        'stars': df.loc[:,3]+1,
                                        'star_votes': df.loc[:,4],
                                        'location': [area.split(',')[0] for area in df.loc[:,5]],
                                        'sub_location': [area.split(',')[1] for area in df.loc[:,5]],
                                        'type': [typ.split(',')[0] for typ in df.loc[:,6]]
                                       })
climb_observed_features.to_csv(pickle_string+'_observed_features.csv', index=False)
climb_observed_features.head()

,Climb,location,rating,rating_dif,star_votes,stars,sub_location,type
0,105741011,Idaho,14.5,0,164,3.8,City of Rocks,Trad
1,105822918,Utah,14.5,0,102,4.1,Wasatch Range,Trad
2,105721840,California,17.5,0,80,3.8,Joshua Tree National Park,Trad
3,105858078,International,21.0,0,57,4.2,North America,Sport
4,105732260,Nevada,17.5,0,154,4.1,Red Rock,Trad


#Scratch Work Below

In [13]:
sp_mat = sp.coo_matrix(sp_mat_input, shape=(len(user_ids), len(climb_ids)) )

row2 = []
col2 = []
data2 = []
for key in data_dict:
    data2 += [data_dict[key]]
    row2 += [key[0]]
    col2 += [key[1]]
    
sp_mat2 = sp.coo_matrix((data2,(row2,col2)), shape=(len(user_ids), len(climb_ids)) )


In [25]:
sp_mat2.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 4, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 4, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [26]:
sp_mat2.shape

(699, 940)

In [29]:
i = 0
for el in sp_mat2.toarray().flatten():
    if el >5:
        i += 1
        #print el
print i

0


In [ ]:
[rating_map[yds] for yds in df.loc[:,1] ]

##Matrix Factorization


In [7]:
import numpy as np

In [72]:
def matrix_factorization(R, K=10, steps=5000, alpha=0.0002, beta=0.02, e_input=0.1, P=False, Q=False):
    (N, M) = R.shape
    if P is False and Q is False:
        print N
        print M
        P = np.random.rand(N,K)
        Q = np.random.rand(K,M)
        
        #P = np.ones((N,K))*np.sqrt(3./K)
        #Q = np.ones((K,M))*np.sqrt(3./K)
    e = None
    for step in xrange(steps):
        
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i,j]>0:
                    eij = R[i,j] - np.dot(P[i,:], Q[:,j])
                    for k in xrange(K):
                        P[i,k]=P[i,k]+alpha*(2*eij*Q[k,j]-beta*P[i,k])
                        Q[k,j]=Q[k,j]+alpha*(2*eij*P[i,k]-beta*Q[k,j])
        eR = np.dot(P,Q)
        e = 0
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i,j] > 0:
                    e=e+pow(R[i,j]-np.dot(P[i,:],Q[:,j]),2)
                    for k in xrange(K):
                        e=e+(beta/2)*(pow(P[i,k],2)+pow(Q[k,j],2))
        if e < e_input:
            print 'Reached error limit!'
            break
    return P, Q, e, step

In [65]:
with open('nP.p','r') as f:
    nP = pickle.load(f)
with open('nQ.p','r') as f:
    nQ = pickle.load(f)
        
nP, nQ, e, step = matrix_factorization(sp_mat2.todense(), steps = 5000)
with open('nP.p','w') as f:
    pickle.dump(nP, f)
with open('nQ.p','w') as f:
    pickle.dump(nQ, f)

699
940


In [66]:
np.matrix(nP.dot(nQ))

matrix([[ 3.05667909,  3.        ,  3.        , ...,  3.        ,
          3.        ,  3.        ],
        [ 3.05667909,  3.        ,  3.        , ...,  3.        ,
          3.        ,  3.        ],
        [ 3.05667909,  3.        ,  3.        , ...,  3.        ,
          3.        ,  3.        ],
        ..., 
        [ 3.05667909,  3.        ,  3.        , ...,  3.        ,
          3.        ,  3.        ],
        [ 3.05667909,  3.        ,  3.        , ...,  3.        ,
          3.        ,  3.        ],
        [ 3.05667909,  3.        ,  3.        , ...,  3.        ,
          3.        ,  3.        ]])

In [67]:
e

2.1462808305060759

In [244]:
    with open('monday_ubuntu_P', 'w') as f:
        pickle.dump(nP, f)
    with open('monday_ubuntu_Q', 'w') as f:
        pickle.dump(nQ, f)

In [73]:
R = np.array([
         [5,3,0,1],
         [4,0,0,1],
         [1,1,0,5],
         [1,0,0,4],
         [0,1,5,4],
        ])
P_test, Q_test, e_test, step_test = matrix_factorization(R)
print e_test
P_test.dot(Q_test)


5
4
1.17555644524


array([[ 4.97894996,  2.9777366 ,  3.16219139,  1.00252009],
       [ 3.98109554,  2.29385178,  2.21465519,  1.00089718],
       [ 1.01432272,  0.96373385,  5.29337258,  4.9701693 ],
       [ 0.99355699,  0.76207593,  4.07752647,  3.9818437 ],
       [ 1.87396211,  1.03892373,  4.97739959,  3.99569796]])

In [70]:
R_data = R.flatten()
R_data

array([5, 3, 0, 1, 4, 0, 0, 1, 1, 1, 0, 5, 1, 0, 0, 4, 0, 1, 5, 4])